## 라이브러리 선언

In [10]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import os
import openai
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import UnstructuredFileLoader
from langchain.schema import HumanMessage, SystemMessage, Document
from langchain.memory import ConversationBufferMemory
from langchain.chains import RetrievalQA, ConversationalRetrievalChain
from langchain.prompts import PromptTemplate
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate, FewShotPromptTemplate
from langchain.chains import LLMChain
from langchain.prompts.pipeline import PipelinePromptTemplate

from dotenv import load_dotenv

## API_KEY 선언

In [13]:
load_dotenv()
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
openai.api_key = OPENAI_API_KEY

# Zero-Shot

In [15]:
full_template = """
{intro}
{input_prompt}
 """
full_prompt = PromptTemplate.from_template(full_template)

In [16]:
intro_template = """
너는 cs 담당자야 \n
주어진 text를 분석해서 긍정 리뷰인지 부정 리뷰인지 확인하려고 해 \n
주어지는 리뷰가 긍정이면 '긍정 리뷰입니다', 부정이면 '부정 리뷰압니다' 로 분류해줘
"""
intro_prompt = PromptTemplate.from_template(intro_template)

In [17]:
input_template = """ 
<text> {input} </text>
"""
input_prompt = PromptTemplate.from_template(input_template)

In [18]:
prompts = [
    ("intro", intro_prompt),
    ("input_prompt", input_prompt),
]

In [19]:
pipeline_prompt = PipelinePromptTemplate(
    final_prompt=full_prompt, pipeline_prompts=prompts
)

In [20]:
# txt 형식을 dataframe으로 불러옴
with open('../data/쇼핑/naver_shopping.txt', 'r') as f:
    data = f.readlines()

data_split = [x.strip().split('\t') for x in data[0:]]
df = pd.DataFrame(data_split, columns=['score', 'review'])

In [21]:
# n개 만큼 추출하여 input으로 저장
df_sampled = df.sample(n=20, random_state=42)

formatted_prompts = [pipeline_prompt.format(
    input=row['review']) for index, row in df_sampled.iterrows()]

In [22]:
# 작업을 수행할 chat 선언
chat = OpenAI(openai_api_key=OPENAI_API_KEY)

In [23]:
# 응답을 저장할 response 선언
responses = []
# formatted_prompts의 input에 대한 응답 받아오기 수행
for prompt in formatted_prompts:
    response = chat(prompt)
    responses.append(response)

# 결과 출력
for response in responses:
    print(response)


'부정 리뷰입니다'

긍정 리뷰입니다.


긍정 리뷰입니다.

주어진 리뷰는 긍정 리뷰입니다.

긍정 리뷰입니다

부정 리뷰입니다.
긍정 리뷰입니다

부정 리뷰입니다.

긍정 리뷰입니다.
 '긍정 리뷰입니다'

긍정 리뷰입니다
 부정 리뷰입니다.

'긍정 리뷰입니다'

주어진 리뷰는 부정 리뷰입니다.
 '부정 리뷰입니다'

'부정 리뷰입니다.'
 '긍정 리뷰입니다'

'긍정 리뷰입니다' 

'부정 리뷰입니다.'

'긍정 리뷰입니다' 


In [24]:
# 응답 결과값과 실제 값을 비교하기 위해 컬럼으로 저장
df_sampled['tag'] = df_sampled['score'].apply(lambda x: '긍정' if int(x) >=3 else '부정')
df_sampled['predict'] = responses

In [25]:
def extract_negative(text):
    if '부정' in text:
        return '부정'
    elif '긍정' in text:
        return '긍정'
    else:
        return None

In [26]:
# '긍부정'만 추출하여 새로운 컬럼에 저장
df_sampled['predict'] = df_sampled['predict'].apply(extract_negative)

df_sampled

,score,review,tag,predict
119737,2,마감이 안좋아요...실밥도 많고 바느질도 부족한 부분이 몇군데 있네요...교환받기 ...,부정,부정
72272,5,깨끗하게 잘 다듬어져 있어요. 맛도좋고요.,긍정,긍정
158154,4,재구매 배송빨라요 길냥이들이 잘먹어요~~ 대용량이라 좋네요~,긍정,긍정
65426,5,제품도 빨리 배송해주시고 꼼꼼하게 잘챙겨주셨어요,긍정,긍정
30074,5,기타 남 멋지고 예뻐요 여러 사은품도 좋아요,긍정,긍정
23677,4,기존것보다 다리를 올려놓으면 푹빠지니깐 무서워서 안올라가요.ㅠㅠ 안고 올려놓으면 가...,긍정,부정
134858,2,상품은잘받았습니다 요청한거와 손잡이방향은다르게왔지만 설치는 잘했습니다. 좀더 신경을...,부정,긍정
176418,2,재구매 아기땜에 하루빨리 필요했는데 배송도 오래 걸렸는데 벨크로가 안왔어요 민원처리...,부정,부정
132467,5,좋네요. 사서 방전된 차에 점프 했는데 잘 됐습니다.,긍정,긍정
4082,4,저렴하게 잘샀어요ㅎ 쓰던거라 좋아요,긍정,긍정


In [27]:
df_sampled['is_correct'] = df_sampled['predict'] == df_sampled['tag']
# 정확도 계산
accuracy = df_sampled['is_correct'].mean()
# 결과 출력
print(f"정확도: {accuracy * 100:.2f}%")

정확도: 90.00%


# Few-Shot

## Template 정의

### full_template 
- 전체 템플릿의 형식 정의

    - intro : 초입에 해당하며, 앞으로 어떤 내용이 오고 어떤 결과값을 원하는지 정의
    - example : 예시 데이터 정의
    - input_prompt : 앞의 내용을 바탕으로 결과값을 받기 위한 input 정의

In [28]:
full_template = """
{intro}
{example}
{input_prompt}
 """
full_prompt = PromptTemplate.from_template(full_template)

### intro_template 정의

In [29]:
intro_template = """
너는 cs 담당자야 \n
주어진 text를 분석해서 긍정 리뷰인지 부정 리뷰인지 확인하려고 해 \n
예시 데이터는 <text> 리뷰 </text> <tag> 긍정/부정 리뷰입니다 </tag> 형식의 xml로 주어질거야 \n
예시 데이터를 보고 주어지는 리뷰가 긍정이면 '긍정 리뷰입니다', 부정이면 '부정 리뷰압니다' 로 분류해줘
"""
intro_prompt = PromptTemplate.from_template(intro_template)

### example_template 정의
- text : 리뷰 텍스트
- tag : text에 대한 긍/부정 값

In [30]:
example_template = """ 
<text>'아주좋아요 바지 정말 좋아서2개 더 구매했어요 이가격에 대박입니다. 바느질이 조금 엉성하긴 하지만 편하고 가성비 최고예요.'</text>
<tag>'긍정 리뷰입니다'</tag>
<text>'선물용으로 빨리 받아서 전달했어야 하는 상품이었는데 머그컵만 와서 당황했습니다. 전화했더니 바로주신다했지만 배송도 누락되어있었네요.. 확인안하고 바로 선물했으면 큰일날뻔했네요..이렇게 배송이 오래걸렸으면 사는거 다시 생각했을거같아요 아쉽네요..'</text>
<tag>'부정 리뷰입니다'</tag>
<text>'처음 쓸때만 좋고 쓰다보니 의자뺄때 소리 계속 나요 그래서 붙이는부직포 덧붙였는데 여전히 끌리네요 비추에요 테니스공으로 살껄 이쁜거 산다고 이거 사서 후회되요'</text>
<tag>'부정 리뷰입니다'</tag>
"""
example_prompt = PromptTemplate.from_template(example_template)

### input_template 정의

In [31]:
input_template = """ 
<text> {input} </text>
"""
input_prompt = PromptTemplate.from_template(input_template)

### prompt 정의

In [32]:
prompts = [
    ("intro", intro_prompt),
    ("example", example_prompt),
    ("input_prompt", input_prompt),
]

## prompt pipeline 정의

In [33]:
pipeline_prompt = PipelinePromptTemplate(
    final_prompt=full_prompt, pipeline_prompts=prompts
)

### input_prompt의 input에 넣기 위한 data processing

In [34]:
# txt 형식을 dataframe으로 불러옴
with open('../data/쇼핑/naver_shopping.txt', 'r') as f:
    data = f.readlines()

data_split = [x.strip().split('\t') for x in data[0:]]
df = pd.DataFrame(data_split, columns=['score', 'review'])

In [35]:
# n개 만큼 추출하여 input으로 저장
df_sampled = df.sample(n=20, random_state=42)

formatted_prompts = [pipeline_prompt.format(
    input=row['review']) for index, row in df_sampled.iterrows()]

In [36]:
# 작업을 수행할 chat 선언
chat = OpenAI(openai_api_key=OPENAI_API_KEY)

In [37]:
# 응답을 저장할 response 선언
responses = []
# formatted_prompts의 input에 대한 응답 받아오기 수행
for prompt in formatted_prompts:
    response = chat(prompt)
    responses.append(response)

# 결과 출력
for response in responses:
    print(response)


 <tag> '긍정 리뷰입니다' </tag>
 <tag>'긍정 리뷰입니다'</tag>


<text>'배송도 빨라서 좋았고, 제품 상태도 깔끔하고 만족스러웠어요. 이 가격에 이렇게 좋은 제품을 구매할 수 있어서 너무 좋아요.'</text>
<tag>'긍정 리뷰입니다'</tag>
 <tag>'긍정 리뷰입니다'</tag>
 <tag>'긍정 리뷰입니다'</tag>
 <tag>'부정 리뷰입니다'</tag>
 <tag>'부정 리뷰입니다'</tag>
 <tag>'긍정 리뷰입니다'</tag>
 <tag>'긍정 리뷰입니다'</tag>

<tag>'긍정 리뷰입니다'</tag>


<tag>'부정 리뷰입니다'</tag>
 <tag> '긍정 리뷰입니다'</tag>
 <tag>'부정 리뷰입니다'</tag>
 <tag>'부정 리뷰입니다'</tag>
 <tag>'부정 리뷰입니다'</tag>


<tag>'긍정 리뷰입니다'</tag>
 <tag>'긍정 리뷰입니다'</tag>

<tag>'부정 리뷰입니다'</tag>
 <tag>'긍정 리뷰입니다'</tag>


## 성능 평가

In [38]:
# 응답 결과값과 실제 값을 비교하기 위해 컬럼으로 저장
df_sampled['tag'] = df_sampled['score'].apply(lambda x: '긍정' if int(x) >=3 else '부정')
df_sampled['predict'] = responses

In [39]:
# '긍부정'만 추출하여 새로운 컬럼에 저장
df_sampled['predict'] = df_sampled['predict'].apply(extract_negative)

df_sampled

,score,review,tag,predict
119737,2,마감이 안좋아요...실밥도 많고 바느질도 부족한 부분이 몇군데 있네요...교환받기 ...,부정,None
72272,5,깨끗하게 잘 다듬어져 있어요. 맛도좋고요.,긍정,긍정
158154,4,재구매 배송빨라요 길냥이들이 잘먹어요~~ 대용량이라 좋네요~,긍정,긍정
65426,5,제품도 빨리 배송해주시고 꼼꼼하게 잘챙겨주셨어요,긍정,긍정
30074,5,기타 남 멋지고 예뻐요 여러 사은품도 좋아요,긍정,긍정
23677,4,기존것보다 다리를 올려놓으면 푹빠지니깐 무서워서 안올라가요.ㅠㅠ 안고 올려놓으면 가...,긍정,긍정
134858,2,상품은잘받았습니다 요청한거와 손잡이방향은다르게왔지만 설치는 잘했습니다. 좀더 신경을...,부정,부정
176418,2,재구매 아기땜에 하루빨리 필요했는데 배송도 오래 걸렸는데 벨크로가 안왔어요 민원처리...,부정,부정
132467,5,좋네요. 사서 방전된 차에 점프 했는데 잘 됐습니다.,긍정,긍정
4082,4,저렴하게 잘샀어요ㅎ 쓰던거라 좋아요,긍정,긍정


In [40]:
df_sampled['is_correct'] = df_sampled['predict'] == df_sampled['tag']
# 정확도 계산
accuracy = df_sampled['is_correct'].mean()
# 결과 출력
print(f"정확도: {accuracy * 100:.2f}%")

정확도: 95.00%
